In [1]:
# importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sqlite3

# importing datasets.

retail = pd.read_csv("data/BrandAverageRetailPrice.csv")
details = pd.read_csv("data/BrandDetails.csv")
sales = pd.read_csv("data/BrandTotalSales.csv")
units = pd.read_csv("data/BrandTotalUnits.csv")

#converting all of our months to datetime
retail['Months'] = pd.to_datetime(retail['Months'])
sales['Months'] = pd.to_datetime(sales['Months'])
units['Months'] = pd.to_datetime(units['Months'])

# Retail dataset

col names = Brands, Months, ARP, "vs. Prior Period"

In [2]:
retail.info()
# ARP has 1939 null entries.
# "vs. Prior Period" has 2712 null entries.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27211 entries, 0 to 27210
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Brands            27211 non-null  object        
 1   Months            27211 non-null  datetime64[ns]
 2   ARP               25279 non-null  float64       
 3   vs. Prior Period  24499 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 850.5+ KB


In [3]:
retail.head()

,Brands,Months,ARP,vs. Prior Period
0,#BlackSeries,2020-08-01,15.684913,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,13.611428,NaN
3,#BlackSeries,2021-02-01,11.873182,-0.127705
4,#BlackSeries,2021-03-01,NaN,-1.000000


# sales dataset
- Need to convert total sales into an int or float.

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Months           25279 non-null  datetime64[ns]
 1   Brand            25279 non-null  object        
 2   Total Sales ($)  25279 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 592.6+ KB


In [5]:
# Concert total sales feature into floats
sales['Total Sales ($)'].replace(',','', regex=True, inplace=True)
sales['Total Sales ($)'] = sales['Total Sales ($)'].str[:8]
sales['Total Sales ($)'] = pd.to_numeric(sales['Total Sales ($)'])

In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Months           25279 non-null  datetime64[ns]
 1   Brand            25279 non-null  object        
 2   Total Sales ($)  25279 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 592.6+ KB


# units dataset
need to convert total units into an int or float

In [7]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Brands            27686 non-null  object        
 1   Months            27686 non-null  datetime64[ns]
 2   Total Units       25712 non-null  object        
 3   vs. Prior Period  24935 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 865.3+ KB


In [8]:
units = units.rename(columns={"Brands": "Brand"})
units = units.rename(columns={"vs. Prior Period": "vs. Prior Period (Units)"})

units['Total Units'].replace(',','', regex=True, inplace=True)
units['Total Units'] = units['Total Units'].str[:8].replace(',','')
units['Total Units'] = pd.to_numeric(units['Total Units'])

In [9]:
units.head(10)

,Brand,Months,Total Units,vs. Prior Period (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,715.5328,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466
4,#BlackSeries,2021-03-01,NaN,-1.000000
5,101 Cannabis Co.,2019-11-01,131.0677,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000
7,101 Cannabis Co.,2020-01-01,345.4134,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169


In [10]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Brand                     27686 non-null  object        
 1   Months                    27686 non-null  datetime64[ns]
 2   Total Units               25712 non-null  float64       
 3   vs. Prior Period (Units)  24935 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 865.3+ KB


In [11]:
# Merging the datasets (Except for details)

In [12]:
retail = retail.rename(columns={"Brands": "Brand"})
retail = retail.rename(columns={"vs. Prior Period": "vs. Prior Period (ARP)"})
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27211 entries, 0 to 27210
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Brand                   27211 non-null  object        
 1   Months                  27211 non-null  datetime64[ns]
 2   ARP                     25279 non-null  float64       
 3   vs. Prior Period (ARP)  24499 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 850.5+ KB


# Merging all datasets except for Details

- for ARP, 1932 nulls
- for ARP vs prior, 2712 nulls
- for Units, 1932 nulls
- for Units vs prior, 2813 nulls

In [13]:
retailXunits = pd.merge(retail, units, on=['Brand', 'Months'], how='left')
full = pd.merge(retailXunits, sales, on=['Brand', 'Months'], how='left')
full.head()

,Brand,Months,ARP,vs. Prior Period (ARP),Total Units,vs. Prior Period (Units),Total Sales ($)
0,#BlackSeries,2020-08-01,15.684913,NaN,1616.3390,NaN,25352.130
1,#BlackSeries,2020-09-01,NaN,-1.000000,NaN,-1.000000,NaN
2,#BlackSeries,2021-01-01,13.611428,NaN,715.5328,NaN,9739.423
3,#BlackSeries,2021-02-01,11.873182,-0.127705,766.6691,0.071466,9102.802
4,#BlackSeries,2021-03-01,NaN,-1.000000,NaN,-1.000000,NaN


In [14]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27211 entries, 0 to 27210
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Brand                     27211 non-null  object        
 1   Months                    27211 non-null  datetime64[ns]
 2   ARP                       25279 non-null  float64       
 3   vs. Prior Period (ARP)    24499 non-null  float64       
 4   Total Units               25279 non-null  float64       
 5   vs. Prior Period (Units)  24893 non-null  float64       
 6   Total Sales ($)           25279 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.7+ MB


## Checking to see what happens if we just drop all rows with NaN values
- We lose 4641 rows, which is ~18% of our dataset

In [15]:
tester = full
tester = tester.dropna()
tester.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22570 entries, 3 to 27209
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Brand                     22570 non-null  object        
 1   Months                    22570 non-null  datetime64[ns]
 2   ARP                       22570 non-null  float64       
 3   vs. Prior Period (ARP)    22570 non-null  float64       
 4   Total Units               22570 non-null  float64       
 5   vs. Prior Period (Units)  22570 non-null  float64       
 6   Total Sales ($)           22570 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.4+ MB


# Feature Creation 

In [16]:
units.sort_values(by=['Brand','Months'], inplace=True)

In [17]:
units.head(10)

,Brand,Months,Total Units,vs. Prior Period (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,715.5328,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466
4,#BlackSeries,2021-03-01,NaN,-1.000000
5,101 Cannabis Co.,2019-11-01,131.0677,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000
7,101 Cannabis Co.,2020-01-01,345.4134,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169


In [18]:
sales.describe()

,Total Sales ($)
count,2.527900e+04
mean,4.093728e+05
std,1.596024e+06
min,0.000000e+00
25%,1.390319e+04
50%,6.210080e+04
75%,2.473270e+05
max,4.036351e+07


# Timeseries feature engineering

In [19]:
# starting with a single brand

blah = units[units.Brand == '#BlackSeries'] # where brand == some brand
blah

,Brand,Months,Total Units,vs. Prior Period (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,715.5328,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466
4,#BlackSeries,2021-03-01,NaN,-1.000000


In [20]:
blah = units
blah.loc[:,'Previous Month (Units)'] = blah.loc[:,'Total Units'].shift(+1)
blah.loc[:,'Rolling Average (Units)'] = (blah.loc[:,'Total Units'].shift(+1) + blah.loc[:,'Total Units'].shift(+2) + blah.loc[:,'Total Units'].shift(+3))/3
blah.shape

(27686, 6)

In [21]:
blah.head(50)

,Brand,Months,Total Units,vs. Prior Period (Units),Previous Month (Units),Rolling Average (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN,NaN,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000,1616.3390,NaN
2,#BlackSeries,2021-01-01,715.5328,NaN,NaN,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466,715.5328,NaN
4,#BlackSeries,2021-03-01,NaN,-1.000000,766.6691,NaN
5,101 Cannabis Co.,2019-11-01,131.0677,NaN,NaN,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000,131.0677,NaN
7,101 Cannabis Co.,2020-01-01,345.4134,NaN,NaN,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883,345.4134,NaN
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169,696.6584,NaN


In [22]:
BS = units[units.Brand == '#BlackSeries']


In [23]:
blah.shape

(27686, 6)

In [24]:
l = blah.shape[0]
i = 1
while i < l:
    if blah.iloc[i]['Brand'] != blah.iloc[i-1]['Brand']:
        blah.drop(i, inplace=True)
        l-=1
    i+=1

In [25]:
blah.shape

(26063, 6)

In [26]:
blah.head(50)

,Brand,Months,Total Units,vs. Prior Period (Units),Previous Month (Units),Rolling Average (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN,NaN,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000,1616.3390,NaN
2,#BlackSeries,2021-01-01,715.5328,NaN,NaN,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466,715.5328,NaN
4,#BlackSeries,2021-03-01,NaN,-1.000000,766.6691,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000,131.0677,NaN
7,101 Cannabis Co.,2020-01-01,345.4134,NaN,NaN,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883,345.4134,NaN
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169,696.6584,NaN
10,101 Cannabis Co.,2020-04-01,712.4981,-0.244750,943.3933,661.821700


In [27]:
# creating new dataframe from consumption column
#data_historic = units[['Total Units']]
# inserting new column with yesterday's consumption values
#blah.loc[:,'Previous Month ()'] = blah.loc[:,'Total Units'].shift(+1)
# inserting another column with difference between yesterday and day before yesterday's consumption values.
#blah.loc[:,'Rolling Average'] = (blah.loc[:,'Total Units'].shift(+1) + blah.loc[:,'Total Units'].shift(+2) + blah.loc[:,'Total Units'].shift(+3))/3

#blah

In [28]:
type(blah)

pandas.core.frame.DataFrame

In [29]:
# Generalising the code for the entire units dataset.

brands = units["Brand"].unique()
brands

array(['#BlackSeries', '101 Cannabis Co.', '10x Infused', ..., 'Zoma',
       'Zuma Topicals', 'iKrusher'], dtype=object)

In [30]:
blah

,Brand,Months,Total Units,vs. Prior Period (Units),Previous Month (Units),Rolling Average (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN,NaN,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000,1616.3390,NaN
2,#BlackSeries,2021-01-01,715.5328,NaN,NaN,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466,715.5328,NaN
4,#BlackSeries,2021-03-01,NaN,-1.000000,766.6691,NaN
...,...,...,...,...,...,...
12660,iKrusher,2020-06-01,671.5098,-0.342416,1021.1760,NaN
12661,iKrusher,2020-07-01,NaN,-1.000000,671.5098,NaN
12662,iKrusher,2021-07-01,404.2586,NaN,NaN,NaN
12663,iKrusher,2021-08-01,987.4264,1.442561,404.2586,NaN


In [31]:
brands.shape

(1639,)

In [32]:
np.unique(brands).shape

(1639,)

In [33]:
#blah = pd.DataFrame()

#for brand in brands:
    #print(brand)
    #brand_df = units[units.Brand == brand]
    #brand_df.loc[:,'Previous Month (Units)'] = brand_df.loc[:,'Total Units'].shift(+1)
    #brand_df.loc[:,'Rolling Average (Units)'] = (brand_df.loc[:,'Total Units'].shift(+1) + brand_df.loc[:,'Total Units'].shift(+2) + brand_df.loc[:,'Total Units'].shift(+3))/3
    #blah.append(brand_df)

#blah

# For Sales Dataset

In [34]:
sales.head(10)

,Months,Brand,Total Sales ($)
0,2018-09-01,10x Infused,1711.334
1,2018-09-01,1964 Supply Co.,25475.210
2,2018-09-01,3 Bros Grow,120153.600
3,2018-09-01,3 Leaf,6063.529
4,2018-09-01,350 Fire,631510.000
5,2018-09-01,710 Labs,2065970.000
6,2018-09-01,A&A Craft Inc,5094.305
7,2018-09-01,AA Packaging,2333.339
8,2018-09-01,Absolute Xtracts,5747227.000
9,2018-09-01,Aces Extracts,155523.700


In [35]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Months           25279 non-null  datetime64[ns]
 1   Brand            25279 non-null  object        
 2   Total Sales ($)  25279 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 592.6+ KB


In [36]:
sales.sort_values(by=['Brand','Months'], inplace=True)
sales.head(20)

,Months,Brand,Total Sales ($)
14023,2020-08-01,#BlackSeries,25352.130
17980,2021-01-01,#BlackSeries,9739.423
18784,2021-02-01,#BlackSeries,9102.802
7908,2019-11-01,101 Cannabis Co.,4465.040
9146,2020-01-01,101 Cannabis Co.,11790.660
9765,2020-02-01,101 Cannabis Co.,20266.760
10424,2020-03-01,101 Cannabis Co.,30465.470
11110,2020-04-01,101 Cannabis Co.,23465.650
11799,2020-05-01,101 Cannabis Co.,21348.390
12530,2020-06-01,101 Cannabis Co.,14111.750


In [37]:
sales_df = sales
sales_df.loc[:,'Previous Month (Sales)'] = sales_df.loc[:,'Total Sales ($)'].shift(+1)
# inserting another column with difference between yesterday and day before yesterday's consumption values.
sales_df.loc[:,'Rolling Average (Sales)'] = (sales_df.loc[:,'Total Sales ($)'].shift(+1) + sales_df.loc[:,'Total Sales ($)'].shift(+2) + sales_df.loc[:,'Total Sales ($)'].shift(+3))/3

sales_df.head(25)

,Months,Brand,Total Sales ($),Previous Month (Sales),Rolling Average (Sales)
14023,2020-08-01,#BlackSeries,25352.130,NaN,NaN
17980,2021-01-01,#BlackSeries,9739.423,25352.130,NaN
18784,2021-02-01,#BlackSeries,9102.802,9739.423,NaN
7908,2019-11-01,101 Cannabis Co.,4465.040,9102.802,14731.451667
9146,2020-01-01,101 Cannabis Co.,11790.660,4465.040,7769.088333
9765,2020-02-01,101 Cannabis Co.,20266.760,11790.660,8452.834000
10424,2020-03-01,101 Cannabis Co.,30465.470,20266.760,12174.153333
11110,2020-04-01,101 Cannabis Co.,23465.650,30465.470,20840.963333
11799,2020-05-01,101 Cannabis Co.,21348.390,23465.650,24732.626667
12530,2020-06-01,101 Cannabis Co.,14111.750,21348.390,25093.170000


In [39]:
sales_df.shape

(25279, 5)

In [40]:
print(sales_df.shape)

l = sales_df.shape[0]
i = 1
while i < l:
    if sales_df.iloc[i]['Brand'] != sales_df.iloc[i-1]['Brand']:
        sales_df.drop(i, inplace=True)
        l-=1
    i+=1

print(sales_df.shape)

(25279, 5)
(23706, 5)


# Brand Feature Engineering

In [42]:
details.head(8)

,State,Channel,Category L1,Category L2,Category L3,Category L4,Category L5,Brand,Product Description,Total Sales ($),...,Total THC,Total CBD,Contains CBD,Pax Filter,Strain,Is Flavored,Mood Effect,Generic Vendor,Generic Items,$5 Price Increment
0,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),"1,103.964857",...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
1,California,Licensed,Inhaleables,Flower,Hybrid,NaN,NaN,#BlackSeries,#BlackSeries - Vanilla Frosting - Flower (Gram),674.645211,...,0,0,THC Only,NaN,Vanilla Frosting,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
2,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"2,473.699102",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$15.00 to $19.99
3,California,Licensed,Inhaleables,Flower,Sativa Dominant,NaN,NaN,#BlackSeries,#BlackSeries - Blueberry Slushy - Flower (Gram),"14,589.916417",...,0,0,THC Only,NaN,Blueberry Slushy,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99
4,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Afghan Kush - Wax,145.39627,...,0,0,THC Only,NaN,Afghan Kush,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$35.00 to $39.99
5,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Skywalker OG - Wax,"3,261.12486",...,0,0,THC Only,NaN,Skywalker OG,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$30.00 to $34.99
6,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Skywalker OG - Wax,"2,062.231412",...,0,0,THC Only,NaN,Skywalker OG,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$20.00 to $24.99
7,California,Licensed,Inhaleables,Concentrates,Dabbable Concentrates,Wax,NaN,101 Cannabis Co.,101 Cannabis Co. - Indica Strain Blends - Wax,62.556665,...,0,0,THC Only,NaN,Indica Strain Blends,NaN,Not Mood Specific,Non-Generic Vendors,Non-Generic Items,$10.00 to $14.99


In [38]:
# brand_df['Category L1'].value_counts()